In [2]:
import pyspark
from pyspark.sql import SparkSession

In [9]:
source_url = 'https://drive.google.com/uc?export=download&id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ'
!wget --no-check-certificate -O multi_source_demo.zip $source_url

In [11]:
!gzip multi_source_demo.zip.gz

gzip: multi_source_demo.zip.gz already has .gz suffix -- unchanged
